# 1. Imports, Installs & Downloads

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker
!pip install roboflow

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.2 MB/s eta 0:00:00
  Crea

In [ ]:
from roboflow import Roboflow
from google.colab import files
import os
import shutil
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector



TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 



In [ ]:
rf = Roboflow(api_key="")
project = rf.workspace("brian-mortimer-mk3tc").project("road-traffic-sign-dataset")
dataset = project.version(5).download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Road-Traffic-Sign-Dataset-5 in coco:: 100%|██████████| 7057/7057 [00:02<00:00, 2744.51it/s]


# 2. Convert Roboflow COCO format to COCO

In [ ]:
!rm -rf dataset
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/test
!mkdir dataset/val
!mkdir dataset/train/images
!mkdir dataset/test/images
!mkdir dataset/val/images
dataset_path = "dataset"
train_path = "dataset/train"
test_path = "dataset/test"
val_path = "dataset/val"

In [ ]:
  def move_robo_files(src_dir, dst_dir, name):
    for file_name in os.listdir(src_dir):
      if file_name.endswith(".jpg") or file_name.endswith(".png"):
        # move to dataset
        src = os.path.join(src_dir, file_name)
        dst = os.path.join(dst_dir, f"images/{file_name}")
        shutil.copy(src, dst)

      elif file_name.endswith(".json"):
        # move to annotations
        src = os.path.join(src_dir, file_name)
        dst = os.path.join(dst_dir, "labels.json")
        shutil.copy(src, dst)
      else:
        print(f"Unassigned File: {file_name}")

In [ ]:
robo_train = os.path.join(dataset.location, "train")
robo_test = os.path.join(dataset.location, "test")
robo_val = os.path.join(dataset.location, "valid")

move_robo_files(robo_test, test_path, "test")
move_robo_files(robo_val, val_path, "val")
move_robo_files(robo_train, train_path, "train")

In [ ]:
# Remove the Class 0 label
train_labels = os.path.join(train_path, "labels.json")
test_labels = os.path.join(test_path, "labels.json")
val_labels = os.path.join(val_path, "labels.json")

for labels in [train_labels, test_labels, val_labels]:
  with open(labels, 'r') as file:
      data = json.load(file)

  data['categories'] = [category for category in data['categories'] if category['id'] != 0]

  with open(labels, 'w') as file:
      json.dump(data, file)

In [ ]:
with open(val_labels, "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

1: 100 km-h
2: 50 km-h
3: 60 km-h
4: 80 km-h
5: Chevron Left
6: Chevron Right
7: Children Crossing
8: Crossroads
9: Cul-de-sac
10: Dangerous Bend Ahead
11: Dangerous Bends Ahead
12: Go Left
13: Keep Left
14: Keep Right
15: No Right Turn
16: No entry
17: No overtaking
18: Pedestrian Crossing
19: Road Narrows
20: Roadworks ahead
21: Roundabout Ahead
22: Side Road on left
23: Side Road on right
24: Speed Ramp Ahead
25: Staggered Cross Roads
26: Stop
27: T-Junction
28: Two-Way Traffic
29: Yield


# 3. Load Dataset

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_path, cache_dir="/tmp/od_data/train",)
test_data = object_detector.Dataset.from_coco_folder(test_path, cache_dir="/tmp/od_data/test")
val_data = object_detector.Dataset.from_coco_folder(val_path, cache_dir="/tmp/od_data/val")
print("train_data size: ", train_data.size)
print("test_data size: ", test_data.size)
print("val_data size: ", val_data.size)

train_data size:  4991
test_data size:  235
val_data size:  470


# 4. Model Fitting

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_V2_I320
hparams = object_detector.HParams(export_dir='exported_model', epochs=30, learning_rate=0.2, batch_size=16)
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
    )

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options
    )

Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.


Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i320
Model: "retina_net_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 80, 80, 24   2257984   
                             ),                                  
                              '3': (None, 40, 40, 32             
                             ),                                  
                              '4': (None, 20, 20, 96             
                             ),                                  
                              '5': (None, 10, 10, 32             
                             0),                                 
                              '6': (None, 10, 10, 12             
                             80)}                                
                                                                 
 fpn_2 (FPN)                 {'5': (None

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


311/311 [==============================] - 119s 246ms/step - total_loss: 4.1216 - cls_loss: 3.7848 - box_loss: 0.0056 - model_loss: 4.0648 - val_total_loss: 1.3461 - val_cls_loss: 1.1268 - val_box_loss: 0.0033 - val_model_loss: 1.2894
Epoch 2/30
311/311 [==============================] - 63s 202ms/step - total_loss: 1.0675 - cls_loss: 0.8259 - box_loss: 0.0037 - model_loss: 1.0108 - val_total_loss: 0.9092 - val_cls_loss: 0.7223 - val_box_loss: 0.0026 - val_model_loss: 0.8524
Epoch 3/30
311/311 [==============================] - 65s 208ms/step - total_loss: 0.7760 - cls_loss: 0.5640 - box_loss: 0.0031 - model_loss: 0.7192 - val_total_loss: 0.7356 - val_cls_loss: 0.5517 - val_box_loss: 0.0025 - val_model_loss: 0.6788
Epoch 4/30
311/311 [==============================] - 61s 196ms/step - total_loss: 0.6199 - cls_loss: 0.4261 - box_loss: 0.0027 - model_loss: 0.5630 - val_total_loss: 0.6548 - val_cls_loss: 0.4747 - val_box_loss: 0.0025 - val_model_loss: 0.5979
Epoch 5/30
311/311 [==========

In [ ]:
model.summary()

Model: "object_detector_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 retina_net_model_2 (Retina  multiple                  2588338   
 NetModel)                                                       
                                                                 
Total params: 2588346 (9.87 MB)
Trainable params: 2545010 (9.71 MB)
Non-trainable params: 43336 (169.28 KB)
_________________________________________________________________


In [ ]:
loss, coco_metrics = model.evaluate(test_data, batch_size=8)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


30/30 [==============================] - 4s 70ms/step - total_loss: 0.4657 - cls_loss: 0.2934 - box_loss: 0.0023 - model_loss: 0.4074


Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.


creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.76s).
Accumulating evaluation results...
DONE (t=0.31s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.343
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.644
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.794
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.366
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.402
 A

In [ ]:
model.export_model()
files.download('exported_model/model.tflite')

Exporting a floating point model


Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>